In [1]:
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

In [2]:
# basic
import numpy as np
import pandas as pd
import pymongo
# visual
import matplotlib.pyplot as plt
%matplotlib inline
#time
import datetime 

#talib
import talib
from talib import abstract

#gui
import tkinter as tk
from  tkinter import ttk  
from tkinter import messagebox
import requests
import re

In [3]:
import io
import requests
import datetime


def crawl_price(stock_id):
    now = int(datetime.datetime.now().timestamp())+86400
    url = "https://query1.finance.yahoo.com/v7/finance/download/" + stock_id + "?period1=0&period2=" + str(now) + "&interval=1d&events=history&crumb=hP2rOschxO0"

    response = requests.post(url)
    
    f = io.StringIO(response.text)
    df = pd.read_csv(f, index_col='Date', parse_dates=['Date'] )
    return df

In [4]:
from suds.client import Client
from suds.xsd.doctor import Import, ImportDoctor
url="http://61.220.30.176/WebOrder/GVETransacs.asmx?WSDL"
imp=Import('http://www.w3.org/2001/XMLSchema',location='http://www.w3.org/2001/XMLSchema.xsd')
imp.filter.add('http://tempuri.org/')
client = Client(url, doctor=ImportDoctor(imp))
print(client)


Suds ( https://github.com/cackharot/suds-py3 )  version: 1.3.3.0 IN  build: 20170311

Service ( GVETransacs ) tns="http://tempuri.org/"
   Prefixes (1)
      ns0 = "http://tempuri.org/"
   Ports (2):
      (GVETransacsSoap)
         Methods (248):
            AdjustOrder(xs:string UserName, xs:string GameCode, xs:string CompCode, xs:string Price, xs:string Volume, xs:string BSAction, xs:string OrderType, xs:string ExpireDate, xs:string OrgVolume, xs:string OrderSource, xs:string CombineNo, )
            CancelOrder(xs:string OrderID, )
            CancelOrderStr_NS(xs:string TokenString, xs:string OrderID, )
            CancelOrder_NS(xs:string TokenString, xs:string OrderID, )
            CancelOrder_NS_STWS(xs:int GMRID, xs:string OrderID, xs:string comment, )
            ChangeOrder(xs:string OrderID, xs:double Volume, )
            ChangeOrderPrice(xs:int gmrid, xs:string orderno, xs:string ordertype, xs:string ordernote, xs:float price, xs:string Lang, )
            ChangeOrder_N

## 第一種

In [5]:
#9日 K值低於 20% 買進，高於 80% 賣出
def testKD92080set(start_money,close,data):
    K = data.iloc[0:,1] #取 K 值
    #D = data.iloc[0:,2] #取 D 值
    #DIF = data2.iloc[0:,0] #取 DIF 值
    #DEA = data2.iloc[0:,1] #取 DEA 值
    #MACD = data2.iloc[0:,2] #取 MACD 值
    #close = df.Close['2018'] #取每日收盤價
    date = data.iloc[0:,0] #取日期
    principal = float(start_money)
    stock_count = 0 #股票張數
    bought_money = 0 #剩餘金額
    final_money = 0 #最終金額
    profit = 0 #獲利
    rateOfReturn = 0.0 #報酬率
    record = [] #存取購買股票時的股價
    lastROP = [] #存取報酬率

    try:
        
        for i in range(len(K)):
            print(date[i])
            print(i)
            print(close[i])
            if(stock_count==0):
                if(float(start_money)>=close[i]*1000):
                    if(K[i]<20):
                        print('K值為:',K[i])
                        print("買了一張股票，股價為:",close[i])
                        stock_count = stock_count + 1
                        start_money = float(start_money) - close[i]*1000
                        record.append(close[i]) #將買的股價放入recorde陣列裡
                        print(record)
                        print('剩餘資金為:',start_money)
                        print('擁有'+str(stock_count)+ '張股票')
                        table.insert("","end",values=("BUY",date[i],close[i],str(stock_count),start_money))
                        #"買/賣 狀態","交易日期","股票價格","股票庫存","資金餘額" 
                    else:
                        print("尚未達到買點")
                        #continue
                else:
                    print("錢不購買")
                    #continue
            else:
                if(close[i] >= record[-1]+5): #現在的股價 >= 停利條件
                    print('現在的股價=',close[i])
                    print('條件: 現股>=最後一次購買股價+10 = ') # recode[-1]為最後一次交易的股價
                    print('最後一次購買股價',record[-1])
                    print('最後一次購買股價+5',record[-1]+5)
                    print('已達停利標準')
                    final_money = start_money + stock_count*close[i]*1000
                    print('賣出'+str(stock_count)+'張股票')
                    record[:] = []
                    stock_count = 0
                    print("股價為:",close[i])
                    print("擁有資金:",final_money)
                    profit = final_money - principal
                    rateOfReturn = profit/principal*100
                    print("獲利:" , profit)
                    print("報酬率:" , rateOfReturn)
                    table.insert("","end",values=("SELL",date[i],close[i],str(stock_count),final_money,profit)) 

                    lastROP.append(rateOfReturn)
                    start_money = final_money #將初始金額改為賣出後的總金額
                    #continue
                elif(close[i] <= record[-1]-5):
                    print('現在的股價=',close[i]) 
                    print('條件: 現股<=最後一次購買股價-5 = ')
                    print('最後一次購買股價',record[-1])
                    print('最後一次購買股價-5',record[-1]-5)
                    print('已達停損標準')
                    final_money = start_money + stock_count*close[i]*1000
                    print('賣出'+str(stock_count)+'張股票')
                    record[:] = []
                    stock_count = 0
                    print("股價為:",close[i])
                    print("i:",i)
                    print("擁有資金:",final_money)
                    profit = final_money - principal
                    rateOfReturn = profit/principal*100
                    print("獲利:" , profit)
                    print("報酬率:" , rateOfReturn)
                    table.insert("","end",values=("SELL",date[i],close[i],str(stock_count),final_money,profit)) 

                    lastROP.append(rateOfReturn)
                    start_money = final_money #將初始金額改為賣出後的總金額
                    #continue
                elif(K[i]<20):
                    if(float(start_money)>=close[i]*1000):
                        print('K值為:',K[i])
                        print("買了一張股票，股價為:",close[i])
                        stock_count = stock_count + 1
                        start_money = float(start_money) - close[i]*1000
                        record.append(close[i]) #將買的股價放入recorde陣列裡
                        print(record)
                        print('剩餘資金為:',start_money)
                        print('擁有'+str(stock_count)+ '張股票')
                        table.insert("","end",values=("BUY",date[i],close[i],str(stock_count),start_money))
                        #"買/賣 狀態","交易日期","股票價格","股票庫存","資金餘額" 
                    else:
                        print("錢不夠再買一張股票")
                        #continue
                elif(K[i]>80):
                    print('K值為:',K[i])
                    print("手上有"+str(stock_count)+"張股票可賣")
                    final_money = start_money + stock_count*close[i]*1000
                    print('賣出'+str(stock_count)+'張股票')
                    record[:] = []
                    stock_count = 0
                    print("股價為:",close[i])
                    print("i:",i)
                    print("擁有資金:",final_money)
                    profit = final_money - principal
                    rateOfReturn = profit/principal*100
                    print("獲利:" , profit)
                    print("報酬率:" , rateOfReturn)
                    table.insert("","end",values=("SELL",date[i],close[i],str(stock_count),final_money,profit)) 

                    lastROP.append(rateOfReturn)
                    start_money = final_money
                else:
                    print("尚未達到買賣點")
                    #continue
        print("\n")        
        print('總報酬率',lastROP[-1])
        #textPanel.insert("end", rateOfReturn)
        textPanel1.insert("end", lastROP[-1])
        return lastROP[-1]
    except:
        tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:金額輸入錯誤')
        return "Fail"

## 第二種

In [6]:
#K值連續三天低於20後反彈(買進)，K值連續三天高於80後反彈(賣出)
def testKDcontSet(start_money,sell,buy,sProfit,sLoss,close,data):
    K = data.iloc[0:,1] #取 K 值
    #close = df.Close['2018'] #取每日收盤價
    date = data.iloc[0:,0] #取日期
    principal = float(start_money)
    stock_count = 0 #股票張數
    #bought_money = 0 #剩餘金額
    final_money = 0 #最終金額
    profit = 0 #獲利
    rateOfReturn = 0.0 #報酬率
    record = [] #存取購買股票時的股價
    lastROP = [] #存取報酬率
    print("2",start_money)
    try:
        for i in range(3,len(K)):
            if stock_count == 0:
                if float(start_money)>=close[i]*1000:
                    if K[i-1]<int(buy) and K[i-2]<int(buy) and K[i-3]<int(buy): 
                        if K[i]>int(buy): # K值連續 3 天小於 20 後上升
                            stock_count = stock_count + 1 #買一張股票
                            start_money = float(start_money) - (float(close[i])*1000) #剩餘資金 = 初始資金 - 當日股價
                            print(date[i-3],'K值為:',K[i-3])
                            print(date[i-2],'K值為:',K[i-2])
                            print(date[i-1],'K值為:',K[i-1])
                            print(date[i],'K值為:',K[i])
                            print("買了一張股票，股價為:",close[i])
                            record.append(close[i]) #將買的股價放入record陣列裡
                            print(record)
                            print('剩餘資金為:',start_money)
                            print('擁有'+str(stock_count)+ '張股票')
                            table.insert("","end",values=("BUY",date[i],close[i],str(stock_count),start_money))
                    else:
                        print("未達買點")
                        continue
                else:
                    print("資金不足")
                    continue

            else:
                if(close[i] >= record[-1]+5): #現在的股價 >= 停利條件
                        print(date[i])
                        print('現在的股價=',close[i]) 
                        print('條件: 現股>=最後一次購買股價+5 = ') # recode[-1]為最後一次交易的股價
                        print('最後一次購買股價',record[-1])
                        print('最後一次購買股價+5',record[-1]+5)
                        print('已達停利標準')
                        final_money = start_money + stock_count*close[i]*1000
                        print('賣出'+str(stock_count)+'張股票')
                        record[:] = []
                        stock_count = 0
                        print("股價為:",close[i])
                        print("擁有資金:",final_money)
                        profit = final_money - principal
                        rateOfReturn = profit/principal*100
                        lastROP.append(rateOfReturn)
                        print("獲利:" , profit)
                        print("報酬率:" , rateOfReturn)
                        table.insert("","end",values=("SELL",date[i],close[i],str(stock_count),final_money,profit)) 

                        #lastROP.append(rateOfReturn)
                        start_money = final_money #將初始金額改為賣出後的總金額
                        continue
                elif(close[i] <= record[-1]-5): #現在的股價 <= 停利條件
                        print(date[i])
                        print('現在的股價=',close[i]) 
                        print('條件: 現股<=最後一次購買股價-5 = ')
                        print('最後一次購買股價',record[-1])
                        print('最後一次購買股價-5',record[-1]-5)
                        print('已達停損標準')
                        final_money = start_money + stock_count*close[i]*1000
                        print('賣出'+str(stock_count)+'張股票')
                        record[:] = []
                        stock_count = 0
                        print("股價為:",close[i])
                        print("擁有資金:",final_money)
                        profit = final_money - principal
                        rateOfReturn = profit/principal*100
                        lastROP.append(rateOfReturn)
                        print("獲利:" , profit)
                        print("報酬率:" , rateOfReturn)
                        table.insert("","end",values=("SELL",date[i],close[i],str(stock_count),final_money,profit)) 

                        #lastROP.append(rateOfReturn)
                        start_money = final_money #將初始金額改為賣出後的總金額
                        continue

                elif K[i-1]<int(buy) and K[i-2]<int(buy) and K[i-3]<int(buy):
                    if K[i]>int(buy):
                        if float(start_money)>=close[i]*1000:
                            stock_count = stock_count + 1 #買一張股票
                            start_money = float(start_money) - (float(close[i])*1000) #剩餘資金 = 初始資金 - 當日股價
                            print(date[i-1],'K值為:',K[i-1])
                            print(date[i-2],'K值為:',K[i-2])
                            print(date[i-3],'K值為:',K[i-3])
                            print(date[i],'K值為:',K[i])
                            print("買了一張股票，股價為:",close[i])
                            record.append(close[i]) #將買的股價放入record陣列裡
                            print(record)
                            print('剩餘資金為:',start_money)
                            print('擁有'+str(stock_count)+ '張股票')
                            table.insert("","end",values=("BUY",date[i],close[i],str(stock_count),start_money))
                        else:
                            print("資金不足")
                elif K[i-1]>int(sell) and K[i-2]>int(sell) and K[i-3]>int(sell):
                    if K[i]<int(sell):
                        print(date[i-1],'K值為:',K[i-1])
                        print(date[i-2],'K值為:',K[i-2])
                        print(date[i-3],'K值為:',K[i-3])
                        print(date[i],'K值為:',K[i])
                        print("手上有"+str(stock_count)+"張股票可賣")
                        final_money = start_money + stock_count*float(close[i])*1000
                        print('賣出'+str(stock_count)+'張股票')
                        record[:]=[]
                        stock_count = 0
                        print("股價為:",close[i])
                        print("擁有資金:",final_money)
                        profit = final_money - principal
                        rateOfReturn = profit/principal*100
                        lastROP.append(rateOfReturn)
                        print("獲利:" , profit)
                        print("報酬率:" , )
                        table.insert("","end",values=("SELL",date[i],close[i],str(stock_count),final_money,profit))
                        #lastROP.append(rateOfReturn)
                        start_money = final_money
                else:
                    print(date[i])
                    print("未達買賣點")
        print("\n")        
        #textPanel.insert("end", lastROP[-1])
        textPanel2.insert("end", lastROP[-1])
        print('總報酬率',lastROP[-1])
        return lastROP[-1] 
    except:
        tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:金額輸入錯誤')
        return "Fail"

## 第三種

In [7]:
## K值向上突破D值 (黃金交叉)，K值向下突破 D值 (死亡交叉)
def testKDcrossSet(start_money,close,data):
    K = data.iloc[0:,1] #取 K 值
    D = data.iloc[0:,2] #取 D 值
    principal = float(start_money)
    #close = df.Close['2018'] #取每日收盤價
    date = data.iloc[0:,0] #取日期
    stock_count = 0 #股票張數
    bought_money = 0.0 #剩餘金額
    final_money = 0.0 #最終金額
    profit = 0.0 #獲利
    rateOfReturn = 0.0 #報酬率
    lastROP = []  
    record = []
    try:  
        for i in range(1,len(K)):
            #print(date[i])
            print(date[i])
            print(i)
            print(close[i])
            if(len(record)==0): #手上沒股票
                if(float(start_money)>=close[i-1]*1000): #身上錢夠
                    if (K[i-1]<D[i-1] and K[i]>D[i]): #黃金交叉，買股票
                            stock_count = stock_count + 1
                            start_money = float(start_money) - (float(close[i-1])*1000)
                            record.append(close[i-1])
                            print("record:",record)
                            print(date[i-1],'K值為:',K[i-1], 'D值為:',D[i-1], date[i], 'K值為:',K[i], 'D值為:',D[i])
                            print("買了一張股票，股價為:",close[i-1])
                            print('剩餘資金為:',start_money)
                            print('擁有'+str(stock_count)+ '張股票')
                            table.insert("","end",values=("BUY",date[i-1],close[i-1],str(stock_count),start_money))
                    else: #不是黃金交叉，直接進入下個迴圈
                        continue
                else: #錢不夠，進入下個迴圈
                    continue
            elif(len(record)!=0): #身上有股票，判斷是否符合停利/停損，是->賣掉，否->判斷為黃金交叉(買)/死亡交叉(賣)/都不是(繼續)
                    if close[i-1] >= record[-1]+5 :#停利，賣股票
                        print(date[i-1],'K值為:',K[i-1], 'D值為:',D[i-1], date[i], 'K值為:',K[i], 'D值為:',D[i])
                        print("手上有"+str(stock_count)+"張股票可賣")
                        final_money = start_money + stock_count*float(close[i-1])*1000
                        print('賣出'+str(stock_count)+'張股票')
                        stock_count = 0
                        record[:]=[]
                        print("股價為:",close[i-1])
                        print("擁有資金:",final_money)
                        profit = final_money - principal
                        rateOfReturn = profit/principal*100
                        print("獲利:" , profit)
                        print("報酬率:" , rateOfReturn)
                        rateOfReturn = round(rateOfReturn, 2)
                        lastROP.append(rateOfReturn)
                        table.insert("","end",values=("SELL",date[i-1],close[i-1],str(stock_count),final_money,profit))
                        start_money = final_money
                    elif close[i-1] <= record[-1]-5 :
                        print(date[i-1],'K值為:',K[i-1], 'D值為:',D[i-1], date[i], 'K值為:',K[i], 'D值為:',D[i])
                        print("手上有"+str(stock_count)+"張股票可賣")
                        final_money = start_money + stock_count*float(close[i-1])*1000
                        print('賣出'+str(stock_count)+'張股票')
                        stock_count = 0
                        record[:]=[]
                        print("股價為:",close[i-1])
                        print("擁有資金:",final_money)
                        profit = final_money - principal
                        rateOfReturn = profit/principal*100
                        print("獲利:" , profit)
                        print("報酬率:" , rateOfReturn)
                        rateOfReturn = round(rateOfReturn, 2)
                        lastROP.append(rateOfReturn)
                        table.insert("","end",values=("SELL",date[i-1],close[i-1],str(stock_count),final_money,profit))
                        start_money = final_money
                    elif (K[i-1]<D[i-1] and K[i]>D[i]):#買
                            if float(start_money)>=close[i]*1000:
                                stock_count = stock_count + 1 #買一張股票
                                bought_money = start_money - (float(close[i-1])*1000) #剩餘資金 = 上一次交易剩餘資金 - 當日股價
                                record.append(close[i])
                                print(date[i-1],'K值為:',K[i-1], 'D值為:',D[i-1], date[i], 'K值為:',K[i], 'D值為:',D[i])
                                print("買了一張股票，股價為:",close[i-1])
                                print('剩餘資金為:',bought_money)
                                print('擁有'+str(stock_count)+ '張股票')
                                table.insert("","end",values=("BUY",date[i-1],close[i-1],str(stock_count),bought_money))
                                start_money = bought_money
                            else:
                                continue
                    elif (K[i-1]>D[i-1] and K[i]<D[i]):
                        print(date[i],'K值為:',K[i-1], 'D值為:',D[i-1], date[i], 'K值為:',K[i], 'D值為:',D[i])
                        print("手上有"+str(stock_count)+"張股票可賣")
                        final_money = start_money + stock_count*float(close[i-1])*1000
                        print('賣出'+str(stock_count)+'張股票')
                        stock_count = 0
                        record[:]=[]
                        print("股價為:",close[i-1])
                        print("擁有資金:",final_money)
                        profit = final_money - principal
                        rateOfReturn = profit/principal*100
                        rateOfReturn = round(rateOfReturn, 2)
                        print("獲利:" , profit)
                        print("報酬率:" , rateOfReturn)
                        lastROP.append(rateOfReturn)
                        print(lastROP)
                        table.insert("","end",values=("SELL",date[i-1],close[i-1],str(stock_count),final_money,profit))
                        start_money = final_money

        textPanel3.insert("end",lastROP[-1])
        return lastROP[-1]
    except:
        tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:金額輸入錯誤')
        return "Fail"

In [26]:
def first():
    stock = stockCode_entry.get()
    start_money = start_money_entry.get()
    #buy = buy_entry.get()
    #sell = sell_entry.get()
    #sProfit = sProfit_entry.get()
    #sLoss = sLoss_entry.get()
    start_time = startTime_entry.get()
    end_time = endTime_entry.get()
    df = crawl_price(stock)
    close = df.Close[start_time:end_time].dropna().astype(float)
    close2 = round(close, 2)
    openp = df.Open[start_time:end_time]
    high = df.High[start_time:end_time]
    low = df.Low[start_time:end_time]
    volumn = df.Volume[start_time:end_time]
    stock = {
    'close':close.dropna().astype(float),
    'open':openp.dropna().astype(float),
    'high':high.dropna().astype(float),
    'low':low.dropna().astype(float),
    'volume': volumn.dropna().astype(float),
    }
    def talib2df(talib_output):
        if type(talib_output) == list:
            ret = pd.DataFrame(talib_output).transpose()
        else:
            ret = pd.Series(talib_output)
        ret.index = stock['close'].index
        return ret;
    
    data = talib2df(talib.abstract.STOCH(stock,fastk_period=9,
                slowk_period=3,
                slowk_matype=0,
                slowd_period=3,
                slowd_matype=0))
        #data.plot()
    data.reset_index(inplace=True)
        #print(data)

        #testKDcrossSet(start_money,close,sProfit,sLoss,data)

    return testKD92080set(start_money,close2,data)


In [29]:
def second():
    stock = stockCode_entry.get()
    start_money = start_money_entry.get()
    buy = buy_entry.get()
    sell = sell_entry.get()
    sProfit = sProfit_entry.get()
    sLoss = sLoss_entry.get()
    start_time = startTime_entry.get()
    end_time = endTime_entry.get()
    df = crawl_price(stock)
    close = df.Close[start_time:end_time].dropna().astype(float)
    close2 = round(close, 2)
    openp = df.Open[start_time:end_time]
    high = df.High[start_time:end_time]
    low = df.Low[start_time:end_time]
    volumn = df.Volume[start_time:end_time]
    stock = {
    'close':close.dropna().astype(float),
    'open':openp.dropna().astype(float),
    'high':high.dropna().astype(float),
    'low':low.dropna().astype(float),
    'volume': volumn.dropna().astype(float),
    }
    
    def talib2df(talib_output):
        if type(talib_output) == list:
            ret = pd.DataFrame(talib_output).transpose()
        else:
            ret = pd.Series(talib_output)
        ret.index = stock['close'].index
        return ret;


    data = talib2df(talib.abstract.STOCH(stock,fastk_period=9,
                slowk_period=3,
                slowk_matype=0,
                slowd_period=3,
                slowd_matype=0))
        #data.plot()
    data.reset_index(inplace=True)
        #print(data)

        #testKDcrossSet(start_money,close,sProfit,sLoss,data)

    return testKDcontSet(start_money,sell,buy,sProfit,sLoss,close2,data)


In [10]:
def third():
    stock = stockCode_entry.get()
    start_money = start_money_entry.get()
    #buy = buy_entry.get()
    #sell = sell_entry.get()
    start_time = startTime_entry.get()
    end_time = endTime_entry.get()
    #sProfit = sProfit_entry.get()
    #sLoss = sLoss_entry.get()
    df = crawl_price(stock)
    close = df.Close[start_time:end_time].dropna().astype(float)
    close2 = round(close, 2)
    openp = df.Open[start_time:end_time]
    high = df.High[start_time:end_time]
    low = df.Low[start_time:end_time]
    volumn = df.Volume[start_time:end_time]
    stock = {
    'close':close.dropna().astype(float),
    'open':openp.dropna().astype(float),
    'high':high.dropna().astype(float),
    'low':low.dropna().astype(float),
    'volume': volumn.dropna().astype(float),
    }
    #callTable()
    def talib2df(talib_output):
        if type(talib_output) == list:
            ret = pd.DataFrame(talib_output).transpose()
        else:
            ret = pd.Series(talib_output)
        ret.index = stock['close'].index
        return ret;
    data = talib2df(talib.abstract.STOCH(stock,fastk_period=9,
                slowk_period=3,
                slowk_matype=0,
                slowd_period=3,
                slowd_matype=0))
        #data.plot()
    data.reset_index(inplace=True)
        #print(data)

        #testKDcrossSet(start_money,close,sProfit,sLoss,data)

    return testKDcrossSet(start_money,close2,data)

In [31]:
def callfunc():
    ans = v.get()
    if(ans=="K值小於設定值進場，大於設定值出場"):
        first()
        #table=ttk.Treeview(frm_r)#表格
    elif(ans=="K值連續3天小於設定值後反彈進場"):
        second()
    elif(ans=="黃金交叉與死亡交叉"):
        third()
def Run3():
    #first()
    #second()
    #third()
    a = int(first())
    b = int(second())
    c = int(third())
    if a>b:
        if b>c:
            print("第一個策略獲利最多")
            bestanswer = "第一個策略獲利最多"
        else:
            if a>c:
                print('第一個策略獲利最多')
                bestanswer = "第一個策略獲利最多"

            else:
                print('第三個策略獲利最多')
                bestanswer = "第三個策略獲利最多"
    elif a<b:
        if b<c:
            print('第三個策略獲利最多')
            bestanswer = "第三個策略獲利最多"
        else:
            if c>a:
                print('第二個策略獲利最多')
                bestanswer = "第二個策略獲利最多"
            else:
                print('第二個策略獲利最多')
                bestanswer = "第二個策略獲利最多"

    answer2 = tk.Label(frm_l, text= bestanswer)
    answer2.grid(row =18, column = 1,sticky='W')
def deletetable():
    x = table.get_children()
    for item in x: 
        table.delete(item) 
    textPanel.delete('1.0','end') 
    textPanel1.delete('1.0','end') 
    textPanel2.delete('1.0','end') 
    textPanel3.delete('1.0','end')

In [12]:
def order():
    win.destroy()
    order = tk.Tk()
    order.title('下單')
    order.geometry("1300x800")
    frame = tk.Frame(order)
    frame.grid(row = 0)
    frame_l = tk.Frame(frame, ) #左視窗
    frame_r = tk.Frame(frame) #右視窗
    frame_c = tk.Frame(frame,)
    frame_l.grid(row = 1,column = 0)
    frame_r.grid(row = 1,column = 2)
    frame_c.grid(row = 1,column = 1)
    #textPanel = tk.Text(frame_r,height=30, width=65)
    #textPanel.grid()
    
    
    
    
    def getOrderRecorde(): #委託紀錄id(

        table=ttk.Treeview(frame_r,show='headings')#表格
        table["columns"]=("委託時間","委託單ID","股票代碼","公司名稱","委託價格","委託數量","買賣類型","掛單類型")
        #表示列,不显示
        table.column("委託時間",width=160)
        table.column("委託單ID",width=100)
        table.column("股票代碼",width=80)
        table.column("公司名稱",width=80)
        table.column("委託價格",width=80)
        table.column("委託數量",width=80)
        table.column("買賣類型",width=80)
        table.column("掛單類型",width=80)

        #显示表头
        table.heading("委託時間",text="委託時間")
        table.heading("委託單ID",text="委託單ID")
        table.heading("股票代碼",text="股票代碼")
        table.heading("公司名稱",text="公司名稱")
        table.heading("委託價格",text="委託價格")
        table.heading("委託數量",text="委託數量")
        table.heading("買賣類型",text="買賣類型")
        table.heading("掛單類型",text="掛單類型")
        
        table.grid(row = 0, column = 0)
        order = client.service.QueryWaitingOrderListGVE3XML_NS("143986D99078C7FA6A0B5BCD8C00ACA4A1DB04385D50EFF4682053C7A0AA4979D7B81BE534ECF4C969E3EA65DFF55137F00EEF9BA3C9FF02C78BA63C37C6202EA694BE201140613AF586FADA1560C84FD5517892C838E79199922D9DDF92DE7626D7BE97ADF465278B4ABD03F7CDF9573B578E7BA64604142854EF2CF90DE997F75D73B2D6499FB20F6841F13751C5C906CB71B300D30C76", "TC", )
        root2 = ET.fromstring(order)
        i = 1
        for country in root2.findall('Order'):
            print("第",i,"筆委託紀錄")
            OrderTime = country.get('OrderTime')
            OrderID = country.get('OrderID')
            AssetID = country.get('AssetID')
            CompName = country.get('CompName')
            Price = country.get('Price')
            Volume = country.get('Volume')
            BSAction = country.get('BSAction')
            OrderType = country.get('OrderType')

            print(OrderTime, OrderID, AssetID, CompName, Price, Volume, BSAction, OrderType)
            i+=1
            table.insert("","end",values=(OrderTime,OrderID,AssetID,CompName,Price,Volume,BSAction,OrderType))
            
        def deletetable(): #清除
            #x = table.get_children()
            #for item in x: 
                #table.delete(item) 
            table.destroy()
            del_btn.destroy()
        del_btn = tk.Button(frame_r, text="清除查詢內容", command=deletetable)
        del_btn.grid(row = 1, column = 0)
        #textPanel.insert("end", order)
    def getInstock(): #庫存
        table=ttk.Treeview(frame_r,show='headings')#表格
        table["columns"]=("股票代碼","公司名稱","持有數量","持有成本","現價","漲跌","盈虧","累積未實現損益")
        #表示列,不显示
        table.column("股票代碼",width=160)
        table.column("公司名稱",width=80)
        table.column("持有數量",width=80)
        table.column("持有成本",width=80)
        table.column("現價",width=80)
        table.column("漲跌",width=80)
        table.column("盈虧",width=80)
        table.column("累積未實現損益",width=80)

        #显示表头
        table.heading("股票代碼",text="股票代碼")
        table.heading("公司名稱",text="公司名稱")
        table.heading("持有數量",text="持有數量")
        table.heading("持有成本",text="持有成本")
        table.heading("現價",text="現價")
        table.heading("漲跌",text="漲跌")
        table.heading("盈虧",text="盈虧")
        table.heading("累積未實現損益",text="累積未實現損益")
        
        table.grid(row = 0, column = 0)
        
        instock = client.service.QueryTodayPositionGve3XML_NS("143986D99078C7FA6A0B5BCD8C00ACA4A1DB04385D50EFF4682053C7A0AA4979D7B81BE534ECF4C969E3EA65DFF55137F00EEF9BA3C9FF02C78BA63C37C6202EA694BE201140613AF586FADA1560C84FD5517892C838E79199922D9DDF92DE7626D7BE97ADF465278B4ABD03F7CDF9573B578E7BA64604142854EF2CF90DE997F75D73B2D6499FB20F6841F13751C5C906CB71B300D30C76", "TC", )
        root1 = ET.fromstring(instock)
        i = 1
        for country in root1.findall('PortfolioAsset'):
            print("第",i,"筆庫存紀錄")
            #LogID = country.get('LogID')
            AssetCode = country.get('AssetCode')
            CompName = country.get('CompName')
            Hold = country.get('Hold')
            Cost = country.get('Cost')
            QuotePrice = country.get('QuotePrice')
            UpDown = country.get('UpDown')
            PL = country.get('PL')
            UnRealizedPL = country.get('UnRealizedPL')
            print(AssetCode, CompName, Hold, Cost, QuotePrice, UpDown, PL, UnRealizedPL)
            i+=1
            table.insert("","end",values=(AssetCode,CompName,Hold,Cost,QuotePrice,UpDown,PL,UnRealizedPL))
        def deletetable(): #清除
            #x = table.get_children()
            #for item in x: 
                #table.delete(item) 
            table.destroy()
            del_btn.destroy()
        del_btn = tk.Button(frame_r, text="清除查詢內容", command=deletetable)
        del_btn.grid(row = 1, column = 0)
        #textPanel.insert("end", instock)
    def getDeal(): #成交紀錄
        table=ttk.Treeview(frame_r,show='headings')#表格
        table["columns"]=("交易時間","股票代碼","公司名稱","買賣類型","掛單類型","成交價","成交量","手續費","交易訊息")
        table.column("交易時間",width=160)   #表示列,不显示
        table.column("股票代碼",width=80)
        table.column("公司名稱",width=80)
        table.column("買賣類型",width=80)
        table.column("掛單類型",width=80)
        table.column("成交價",width=80)
        table.column("成交量",width=80)
        table.column("手續費",width=80)
        table.column("交易訊息",width=80)

        table.heading("交易時間",text="交易時間")  #显示表头
        table.heading("股票代碼",text="股票代碼")
        table.heading("公司名稱",text="公司名稱")
        table.heading("買賣類型",text="買賣類型")
        table.heading("掛單類型",text="掛單類型")
        table.heading("成交價",text="成交價")
        table.heading("成交量",text="成交量")
        table.heading("手續費",text="手續費")
        table.heading("交易訊息",text="交易訊息")
        
        table.grid(row = 0, column = 0)
        deal = client.service.QueryDealLogGVE3ByGMRDayRangeLiteXML_NS(1111708496, "2019/04/01", "2019/12/31", "TW", )
        i = 1
        root = ET.fromstring(deal)
        for country in root.findall('GVEUserLog'):
            print("第",i,"筆成交紀錄")
            #LogID = country.get('LogID')
            LogTime = country.get('LogTime')
            AssetCode = country.get('AssetCode')
            CompName = country.get('CompName')
            BSAction = country.get('BSAction')
            OrderType = country.get('OrderType')
            Price = country.get('Price')
            Volume = country.get('Volume')
            fee = country.get('fee')
            logdesc = country.get('logdesc')
            print(LogTime, AssetCode, CompName, BSAction, OrderType, Price, Volume, fee, logdesc)
            i+=1
            table.insert("","end",values=(LogTime,AssetCode,CompName,BSAction,OrderType,Price,Volume,fee,logdesc))
            
        def deletetable(): #清除
            #x = table.get_children()
            #for item in x: 
                #table.delete(item) 
            table.destroy()
            del_btn.destroy()
        del_btn = tk.Button(frame_r, text="清除查詢內容", command=deletetable)
        del_btn.grid(row = 1, column = 0)
    def getOrder(): #下單
    
            #GMRIDstr = GMRIDstr_Entry.get()
            compcode = CompCode_Entry.get()
            price = Price_Entry.get()
            volume = Volume_Entry.get()
            bsaction = BSAction_Entry.get()
            ordertype = OrderType_Entry.get()
            #currency = Currency_Entry.get()
            isoddlot = IsOddLot_Entry.get()
            ordernote = OrderNote_Entry.get()
            #octype = OCtype_Entry.get()
            #combineno = CombineNo_Entry.get()
            #parameter = OrderParameter_Entry.get()
            #lang = lang_Entry.get()
            #ip = str_ip_Entry.get()

            result = client.service.PutOrderXML3("BCSP", compcode, price, volume, bsaction, ordertype, isoddlot, "TWD", ordernote, "0", "","0", "TC", "127.0.0.1", )
            print(result)
            new = result.split("\"")
            if new[1] == "Success":
                tk.messagebox.showinfo(title='回報', message='Success')
            elif new[1] == "Failure":
                tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:'+new[5])
            else:
                tk.messagebox.showinfo(title='回報',message = 'Failure\n'+'原因:格式輸入錯誤')

    def getPrice():
        table=ttk.Treeview(frame_r,show='headings')#表格
        table["columns"]=("成交價","成交量","委買價","委買量","委賣價","委賣量","昨收價","漲停價","跌停價")
         #表示列,不显示
        table.column("成交價",width=160)
        table.column("成交量",width=80)
        table.column("委買價",width=80)
        table.column("委買量",width=80)
        table.column("委賣價",width=80)
        table.column("委賣量",width=80)
        table.column("昨收價",width=80)
        table.column("漲停價",width=80)
        table.column("跌停價",width=80)

        table.heading("成交價",text="成交價")  #显示表头
        table.heading("成交量",text="成交量")
        table.heading("委買價",text="委買價")
        table.heading("委買量",text="委買量")
        table.heading("委賣價",text="委賣價")
        table.heading("委賣量",text="委賣量")
        table.heading("昨收價",text="昨收價")
        table.heading("漲停價",text="漲停價")
        table.heading("跌停價",text="跌停價")
        
        table.grid(row = 0, column = 0)
        compcode = CompCode_Entry.get()
        price = client.service.QueryQuote5Price(compcode, )
        r = ET.fromstring(price)
        r_new = "您查詢的股價為:\n"+"成交價:"+str(r[0][0].text)+\
                                "\n"+"成交量:"+str(r[0][1].text)+\
                                "\n"+"委買價:"+str(r[0][2].text)+\
                                "\n"+"委買量:"+str(r[0][22].text)+\
                                "\n"+"委賣價:"+str(r[0][12].text)+\
                                "\n"+"委賣量:"+str(r[0][23].text)+\
                                "\n"+"昨收價:"+str(r[0][24].text)+\
                                "\n"+"漲停價:"+str(r[0][25].text)+\
                                "\n"+"跌停價:"+str(r[0][26].text)+"\n"
        
        
        table.insert("","end", values=(str(r[0][0].text),str(r[0][1].text),str(r[0][2].text),str(r[0][22].text),str(r[0][12].text),str(r[0][23].text),str(r[0][24].text),str(r[0][25].text),str(r[0][26].text)))
        def deletetable(): #清除
            #x = table.get_children()
            #for item in x: 
                #table.delete(item) 
            table.destroy()
            del_btn.destroy()
        del_btn = tk.Button(frame_r, text="清除查詢內容", command=deletetable)
        del_btn.grid(row = 1, column = 0)
        
    def cancelOrder():
        cancle_id = cancle_entry.get()
        TokenString = "143986D99078C7FA6A0B5BCD8C00ACA4A1DB04385D50EFF4682053C7A0AA4979D7B81BE534ECF4C969E3EA65DFF55137F00EEF9BA3C9FF02C78BA63C37C6202EA694BE201140613AF586FADA1560C84FD5517892C838E79199922D9DDF92DE7626D7BE97ADF465278B4ABD03F7CDF9573B578E7BA64604142854EF2CF90DE997F75D73B2D6499FB20F6841F13751C5C906CB71B300D30C76"
        client.service.CancelOrderStr_NS(TokenString, cancle_id, )
        getOrderRecorde()
    #GMRIDstr
    #GMRIDstr = tk.Label(frame_l, text="GMRIDstr")
    #GMRIDstr.grid(row=0,column=0)
    #股票代碼
    CompCode = tk.Label(frame_l, text="CompCode")
    CompCode.grid(row=1,column=0)
    #價格
    Price = tk.Label(frame_l, text="Price")
    Price.grid(row=2,column=0)
    #成交量
    Volumn = tk.Label(frame_l, text="Volume")
    Volumn.grid(row=3,column=0)
    #使用代碼 普通買進(B),普通賣出(S),融資買入(MB),融資賣出(RB),融券買入(RS),
    BSAction = tk.Label(frame_l, text="BSAction")
    BSAction.grid(row=4,column=0)
    #下單類型 漲停價(LMTU),跌停價(LMTD),限價單(LMT)，期權市價單：MKT
    OrderType = tk.Label(frame_l, text="OrderType")
    OrderType.grid(row=5,column=0)
    #預設給零  0: normal (照ASSET CATALOG中的交易時間), 1:for TW 零股交易
    IsOddLot = tk.Label(frame_l, text="IsOddLot")
    IsOddLot.grid(row=7,column=0)
    #Currency TWD
    #Currency = tk.Label(frame_l, text="Currency")
    #Currency.grid(row=6,column=0)
    #預設ROD
    OrderNote = tk.Label(frame_l, text ="orderNote")
    OrderNote.grid(row=8,column=0)
    #現貨固定給零；期貨作多：buy，作空：sell；選擇權新倉：O，平倉：C
    #OCtype = tk.Label(frame_l, text ="OCtype")
    #OCtype.grid(row=9,column=0)
    #固定給空白
    #CombineNo = tk.Label(frame_l, text ="CombineNo")
    #CombineNo.grid(row=10,column=0)
    #預設給零  0:普通, 50:當沖單, 60:組合單
    #OrderParameter = tk.Label(frame_l, text ="OrderParameter")
    #OrderParameter.grid(row=11,column=0)
    #Lang //固定給TC
    #lang = tk.Label(frame_l, text ="lang")
    #lang.grid(row=12,column=0)
    #str_ip //IP位置
    #str_ip = tk.Label(frame_l, text ="str_ip")
    #str_ip.grid(row=13,column=0)
    #取消委託
    cancleOrder = tk.Label(frame_l, text = "--------取消委託--------")
    cancleOrder.grid(row=14,column=0)
    cancle = tk.Label(frame_l, text = "輸入ID:")
    cancle.grid(row = 15, column = 0)
    cancle_entry = tk.Entry(frame_l)
    cancle_entry.grid(row=15,column = 1)
    #lb.pack()
    #GMRIDstr = tk.StringVar(value="BCSP")
    CompCode = tk.StringVar(value="2330.tw")
    #Price = tk.StringVar(value="2330.tw")
    Volume = tk.StringVar(value="1000")
    BSAction = tk.StringVar(value="B")
    OrderType = tk.StringVar(value="MKT")
    #Currency = tk.StringVar(value="TWD")
    IsOddLot = tk.StringVar(value="0")
    OrderNote = tk.StringVar(value="ROD")
    #OCtype = tk.StringVar(value="0")
    #OrderParameter = tk.StringVar(value="0")
    #lang = tk.StringVar(value="TC")
    #str_ip = tk.StringVar(value="127.0.0.1")
    
    #GMRIDstr_Entry = tk.Entry(frame_l,textvariable=GMRIDstr)
    #GMRIDstr_Entry.grid(row=0,column=1)

    CompCode_Entry = tk.Entry(frame_l,textvariable=CompCode)
    CompCode_Entry.grid(row=1,column=1)

    Price_Entry = tk.Entry(frame_l,textvariable=Price)
    Price_Entry.grid(row=2,column=1)

    Volume_Entry = tk.Entry(frame_l,textvariable=Volume)
    Volume_Entry.grid(row=3,column=1)

    BSAction_Entry = tk.Entry(frame_l,textvariable=BSAction)
    BSAction_Entry.grid(row=4,column=1)

    OrderType_Entry = tk.Entry(frame_l,textvariable=OrderType)
    OrderType_Entry.grid(row=5,column=1)

    #Currency_Entry = tk.Entry(frame_l,textvariable=Currency)
    #Currency_Entry.grid(row=6,column=1)

    IsOddLot_Entry = tk.Entry(frame_l,textvariable=IsOddLot)
    IsOddLot_Entry.grid(row=7,column=1)

    OrderNote_Entry = tk.Entry(frame_l,textvariable=OrderNote)
    OrderNote_Entry.grid(row=8,column=1)

    #OCtype_Entry = tk.Entry(frame_l,textvariable=OCtype)
    #OCtype_Entry.grid(row=9,column=1)

    #CombineNo_Entry = tk.Entry(frame_l,textvariable=CombineNo)
    #CombineNo_Entry.grid(row=10,column=1)

    #OrderParameter_Entry = tk.Entry(frame_l,textvariable=OrderParameter)
    #OrderParameter_Entry.grid(row=11,column=1)

    #lang_Entry = tk.Entry(frame_l,textvariable=lang)
    #lang_Entry.grid(row=12,column=1)

    #str_ip_Entry = tk.Entry(frame_l,textvariable=str_ip)
    #str_ip_Entry.grid(row=13,column=1)

#-------------------------Button-------------------------#    
    order_btn = tk.Button(frame_c , text="下單", command=getOrder) #彈出視窗表示成功或失敗
    order_btn.grid()
        
    record_btn = tk.Button(frame_c, text="委託紀錄", command=getOrderRecorde)#以表格方式呈現
    record_btn.grid()
    
    instock_btn = tk.Button(frame_c, text="庫存查詢", command=getInstock) #以表格方式呈現
    instock_btn.grid()
    
    deal_btn = tk.Button(frame_c, text="成交查詢", command = getDeal)
    deal_btn.grid()
    
    stockprice_btn = tk.Button(frame_c, text="即時股價查詢", command=getPrice) #彈出視窗，輸入股票代碼，印出股價
    stockprice_btn.grid()
    
    cancle_btn = tk.Button(frame_l, text="取消委託", command = cancelOrder)
    cancle_btn.grid(row = 17,column = 0)

In [36]:
win = tk.Tk() #建立主視窗
win.title("股票回測") #視窗標題
win.geometry("1000x600")#寬*高
#style = ttk.Style(win)
#style.theme_use('classic') 
#win.resizable(False, False)#限制縮放
#win.config(background="#323232") #視窗背景顏色
#框架
frm = tk.Frame(win)
frm.grid(row = 0)
frm_l = tk.Frame(frm, ) #左視窗
frm_r = tk.Frame(frm) #右視窗
frm_l.grid(row = 1,column = 0)
frm_r.grid(row = 1,column = 1)
#tk.Label(frm,bg="#323232",fg="white",text="輸入股票代碼:" )
#-----------------------Text---------------------------------
#股票代碼
stockCode = tk.Label(frm_l, text="輸入股票代碼:")
stockCode.grid(row=0,column=0,sticky='NW')
#股票日期
startTime = tk.Label(frm_l, text="輸入起始時間:")
startTime.grid(row=1,column=0,sticky='NW')
endTime = tk.Label(frm_l, text='輸入結束時間')
endTime.grid(row=2,column=0,sticky='NW')
#初始金額
start_money = tk.Label(frm_l, text="輸入初始金額:")
start_money.grid(row=3,column=0,sticky='NW')
#低於幾%買進
buy = tk.Label(frm_l, text="輸入買進條件:")
buy.grid(row=4,column=0,sticky='NW')
#高於幾%賣出
sell = tk.Label(frm_l, text="輸入賣出條件:")
sell.grid(row=5,column=0,sticky='NW')
#停利
sProfit = tk.Label(frm_l, text="輸入停利條件:")
sProfit.grid(row=6,column=0,sticky='NW')
#停損
sLoss = tk.Label(frm_l, text="輸入停損條件:")
sLoss.grid(row=7,column=0,sticky='NW')
#lb.pack()

#-------------------------table------------------------------
table=ttk.Treeview(frm_r,show='headings')#表格
table["columns"]=("買/賣 狀態","交易日期","股票價格","股票庫存","資金餘額","賣出獲利")
table.column("買/賣 狀態",width=100)   #表示列,不显示
table.column("交易日期",width=100)
table.column("股票價格",width=100)
table.column("股票庫存",width=100)
table.column("資金餘額",width=100)
table.column("賣出獲利",width=100)

table.heading("買/賣 狀態",text="買/賣 狀態")  #显示表头
table.heading("交易日期",text="交易日期")
table.heading("股票價格",text="股票價格")
table.heading("股票庫存",text="股票庫存")
table.heading("資金餘額",text="資金餘額")
table.heading("賣出獲利",text="賣出獲利")
table.grid()
#--------------------------textPanel-----------------------------------
#報酬率
pOfrate = tk.Label(frm_l, text=' 報酬率 :')
pOfrate.grid(row =14, column = 0,sticky='W')
textPanel = tk.Text(frm_l,height=1, width=4)
textPanel.grid(row =14,column=1,sticky='W')
#報酬率 1
ROP1 = tk.Label(frm_l, text=' K值大小')
ROP1.grid(row =15, column = 0,sticky='W')
textPanel1 = tk.Text(frm_l,height=1, width=4)
textPanel1.grid(row =15, column = 1,sticky='W')
ROP2 = tk.Label(frm_l, text=' 連三反彈')
ROP2.grid(row =16, column = 0,sticky='W')
textPanel2 = tk.Text(frm_l,height=1, width=4)
textPanel2.grid(row =16, column = 1,sticky='W')
ROP3 = tk.Label(frm_l, text=' 黃金交叉')
ROP3.grid(row =17, column = 0,sticky='W')
textPanel3 = tk.Text(frm_l, height=1, width=4)
textPanel3.grid(row =17, column = 1,sticky='W')
answer = tk.Label(frm_l, text=' 比較結果:')
answer.grid(row =18, column = 0,sticky='W')

#----------------------------entry-----------------------------------------
#Entry輸入框
stock = tk.StringVar(value='2330.TW')
time = tk.StringVar(value='2018')
sMoney =tk.StringVar(value='1000000')
bEntry = tk.StringVar(value='20')
sEntry = tk.StringVar(value='80')
pEntry = tk.StringVar(value='0.018')
lEntry = tk.StringVar(value='0.01')

stockCode_entry = tk.Entry(frm_l,textvariable=stock)
stockCode_entry.grid(row=0,column=1,sticky='W')
startTime_entry = tk.Entry(frm_l,textvariable=time)
startTime_entry.grid(row=1,column=1,sticky='W')
endTime_entry = tk.Entry(frm_l)
endTime_entry.grid(row=2,column=1,sticky='W')
start_money_entry = tk.Entry(frm_l,textvariable=sMoney)
start_money_entry.grid(row=3,column=1,sticky='W')
buy_entry = tk.Entry(frm_l,textvariable=bEntry)
buy_entry.grid(row=4,column=1,sticky='W')
sell_entry = tk.Entry(frm_l,textvariable=sEntry)
sell_entry.grid(row=5,column=1,sticky='W')
sProfit_entry = tk.Entry(frm_l,textvariable=pEntry)
sProfit_entry.grid(row=6,column=1,sticky='W')
sLoss_entry = tk.Entry(frm_l,textvariable=lEntry)
sLoss_entry.grid(row=7,column=1,sticky='W')
#option = tk.Option
optionList = ["K值小於設定值進場，大於設定值出場","K值連續3天小於設定值後反彈進場","黃金交叉與死亡交叉"]
v = tk.StringVar()
v.set("選擇條件")
optionmenu = tk.OptionMenu(frm_l, v, *optionList)#command=func(*optionList))
optionmenu.grid(row=8,column=0,sticky='W')
#en.pack()

#-------------------------Button-------------------------#
btn = tk.Button(frm_l, text="回測",command=callfunc) #button內容、顏色
#OR獨立寫 btn.config(bg = "skyblue")
#btn.config(width=5, height=5) #button大小
#btn.config(image=img) #不可跟上兩行同時，也就是說圖片就是button
#btn.config(command=say_hi)
btn.grid(row = 11,column = 0,pady = 10) #封裝
btn2 = tk.Button(frm_l, text="清除", command=deletetable) #button內容、顏色
#OR獨立寫 btn.config(bg = "skyblue")
#btn.config(width=5, height=5) #button大小
#btn.config(image=img) #不可跟上兩行同時，也就是說圖片就是button
#btn.config(command=say_hi)
btn2.grid(row = 12,column = 0, pady = 10) #封裝

btn3 = tk.Button(frm_l, text="下單", command=order)
btn3.grid(row = 11,column = 1, pady = 10)



btn4 = tk.Button(frm_l, text="Run3", command = Run3)
btn4.grid(row = 12,column = 1, pady = 10)        

win.mainloop()

2018-01-02 00:00:00
0
232.5
尚未達到買點
2018-01-03 00:00:00
1
237.0
尚未達到買點
2018-01-04 00:00:00
2
239.5
尚未達到買點
2018-01-05 00:00:00
3
240.0
尚未達到買點
2018-01-08 00:00:00
4
242.0
尚未達到買點
2018-01-09 00:00:00
5
242.0
尚未達到買點
2018-01-10 00:00:00
6
236.5
尚未達到買點
2018-01-11 00:00:00
7
235.0
尚未達到買點
2018-01-12 00:00:00
8
237.0
尚未達到買點
2018-01-15 00:00:00
9
240.0
尚未達到買點
2018-01-16 00:00:00
10
240.5
尚未達到買點
2018-01-17 00:00:00
11
242.0
尚未達到買點
2018-01-18 00:00:00
12
248.5
尚未達到買點
2018-01-19 00:00:00
13
255.5
尚未達到買點
2018-01-22 00:00:00
14
261.5
尚未達到買點
2018-01-23 00:00:00
15
266.0
尚未達到買點
2018-01-24 00:00:00
16
258.0
尚未達到買點
2018-01-25 00:00:00
17
258.0
尚未達到買點
2018-01-26 00:00:00
18
255.0
尚未達到買點
2018-01-29 00:00:00
19
258.5
尚未達到買點
2018-01-30 00:00:00
20
253.0
尚未達到買點
2018-01-31 00:00:00
21
255.0
尚未達到買點
2018-02-01 00:00:00
22
259.5
尚未達到買點
2018-02-02 00:00:00
23
259.5
尚未達到買點
2018-02-05 00:00:00
24
253.0
尚未達到買點
2018-02-06 00:00:00
25
239.0
尚未達到買點
2018-02-07 00:00:00
26
240.0
K值為: 19.00112233445567
買了一張股票，股價為: 240.0
[240

尚未達到買點
2019-02-18 00:00:00
271
230.0
尚未達到買點
2019-02-19 00:00:00
272
229.0
尚未達到買點
2019-02-20 00:00:00
273
234.5
尚未達到買點
2019-02-21 00:00:00
274
236.5
尚未達到買點
2019-02-22 00:00:00
275
236.5
尚未達到買點
2019-02-25 00:00:00
276
238.0
尚未達到買點
2019-02-26 00:00:00
277
239.5
尚未達到買點
2019-02-27 00:00:00
278
239.0
尚未達到買點
2019-03-04 00:00:00
279
235.5
尚未達到買點
2019-03-05 00:00:00
280
233.0
尚未達到買點
2019-03-06 00:00:00
281
234.0
尚未達到買點
2019-03-07 00:00:00
282
234.0
尚未達到買點
2019-03-08 00:00:00
283
230.0
尚未達到買點
2019-03-11 00:00:00
284
230.5
K值為: 17.696969696969663
買了一張股票，股價為: 230.5
[230.5]
剩餘資金為: 881000.0
擁有1張股票
2019-03-12 00:00:00
285
235.5
現在的股價= 235.5
條件: 現股>=最後一次購買股價+10 = 
最後一次購買股價 230.5
最後一次購買股價+5 235.5
已達停利標準
賣出1張股票
股價為: 235.5
擁有資金: 1116500.0
獲利: 116500.0
報酬率: 11.65
2019-03-13 00:00:00
286
237.0
尚未達到買點
2019-03-14 00:00:00
287
234.5
尚未達到買點
2019-03-15 00:00:00
288
239.0
尚未達到買點
2019-03-18 00:00:00
289
241.0
尚未達到買點
2019-03-19 00:00:00
290
240.5
尚未達到買點
2019-03-20 00:00:00
291
242.0
尚未達到買點
2019-03-21 00:00:00
292


2 1000000
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
2018-02-08 00:00:00 K值為: 18.28282828282828
2018-02-09 00:00:00 K值為: 16.829836829836825
2018-02-12 00:00:00 K值為: 17.76223776223776
2018-02-21 00:00:00 K值為: 26.666666666666668
買了一張股票，股價為: 242.5
[242.5]
剩餘資金為: 757500.0
擁有1張股票
2018-02-22 00:00:00
未達買賣點
2018-02-23 00:00:00
未達買賣點
2018-02-26 00:00:00
未達買賣點
2018-02-27 00:00:00
未達買賣點
2018-03-01 00:00:00
未達買賣點
2018-03-02 00:00:00
未達買賣點
2018-03-05 00:00:00
未達買賣點
2018-03-06 00:00:00
現在的股價= 250.0
條件: 現股>=最後一次購買股價+5 = 
最後一次購買股價 242.5
最後一次購買股價+5 247.5
已達停利標準
賣出1張股票
股價為: 250.0
擁有資金: 1007500.0
獲利: 7500.0
報酬率: 0.75
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
未達買點
2018-05-02 00:00:00 K值為: 14.893617021276595
2018-05-03 00:00:00 K值為: 12.796793092815294
2018-05-04 00:00:00 K值為: 16.786154794942956
2018

149
241.5
2018-08-16 00:00:00
150
239.0
2018-08-17 00:00:00
151
239.5
2018-08-20 00:00:00
152
239.5
2018-08-21 00:00:00
153
241.0
record: [239.5]
2018-08-20 00:00:00 K值為: 13.333333333333355 D值為: 19.268077601410976 2018-08-21 00:00:00 K值為: 20.00000000000002 D值為: 16.97530864197535
買了一張股票，股價為: 239.5
剩餘資金為: 827500.0
擁有1張股票
2018-08-22 00:00:00
154
242.0
2018-08-23 00:00:00
155
244.5
2018-08-24 00:00:00
156
243.5
2018-08-23 00:00:00 K值為: 55.76719576719578 D值為: 35.194003527336896 2018-08-24 00:00:00 K值為: 68.68386243386244 D值為: 51.421957671957706
手上有1張股票可賣
賣出1張股票
股價為: 244.5
擁有資金: 1072000.0
獲利: 72000.0
報酬率: 7.199999999999999
2018-08-27 00:00:00
157
245.0
2018-08-28 00:00:00
158
249.5
2018-08-29 00:00:00
159
259.0
2018-08-30 00:00:00
160
263.5
2018-08-31 00:00:00
161
256.0
2018-09-03 00:00:00
162
257.0
2018-09-04 00:00:00
163
257.5
2018-09-05 00:00:00
164
264.0
record: [257.5]
2018-09-04 00:00:00 K值為: 59.10734463276837 D值為: 69.55178907721283 2018-09-05 00:00:00 K值為: 67.33333333333334 D值為: 64.849

2019-04-17 00:00:00 K值為: 95.45454545454542 D值為: 91.59078469423291 2019-04-18 00:00:00 K值為: 93.74389051808403 D值為: 93.9103011683656
手上有1張股票可賣
賣出1張股票
股價為: 261.5
擁有資金: 1161500.0
獲利: 161500.0
報酬率: 16.150000000000002
2019-04-19 00:00:00
311
264.5
2019-04-22 00:00:00
312
266.0
2019-04-23 00:00:00
313
268.0
2019-04-24 00:00:00
314
269.0
record: [268.0]
2019-04-23 00:00:00 K值為: 81.98198198198195 D值為: 82.72524724137621 2019-04-24 00:00:00 K值為: 89.23660502607868 D值為: 84.2258217470441
買了一張股票，股價為: 268.0
剩餘資金為: 893500.0
擁有1張股票
2019-04-25 00:00:00
315
267.5
2019-04-26 00:00:00
316
260.0
2019-04-26 00:00:00 K值為: 90.33457799905165 D值為: 87.18438833570406 2019-04-26 00:00:00 K值為: 70.04877495462792 D值為: 83.20665265991937
手上有1張股票可賣
賣出1張股票
股價為: 267.5
擁有資金: 1161000.0
獲利: 161000.0
報酬率: 16.1
[-0.65, 0.35, 1.2, 1.95, 1.85, 2.5, 3.0, 3.75, 3.75, 4.25, 4.45, 4.55, 5.05, 5.15, 6.45, 6.95, 6.7, 7.2, 7.85, 8.4, 8.45, 9.4, 10.7, 11.2, 11.5, 12.15, 12.9, 13.1, 13.3, 13.6, 13.7, 14.35, 14.85, 14.9, 15.55, 16.15, 16.1]

...

2017-01-04 00:00:00
1
183.0
2017-01-05 00:00:00
2
183.5
2017-01-06 00:00:00
3
184.0
2017-01-09 00:00:00
4
184.0
2017-01-10 00:00:00
5
184.0
2017-01-11 00:00:00
6
182.0
2017-01-12 00:00:00
7
184.5
2017-01-13 00:00:00
8
181.5
2017-01-16 00:00:00
9
179.5
2017-01-17 00:00:00
10
181.0
2017-01-18 00:00:00
11
181.0
2017-01-19 00:00:00
12
180.5
2017-01-20 00:00:00
13
181.0
2017-01-23 00:00:00
14
185.0
2017-01-24 00:00:00
15
185.5
2017-02-02 00:00:00
16
184.5
2017-02-03 00:00:00
17
184.5
2017-02-06 00:00:00
18
184.5
2017-02-07 00:00:00
19
184.5
2017-02-08 00:00:00
20
183.5
2017-02-09 00:00:00
21
184.0
2017-02-10 00:00:00
22
185.5
2017-02-13 00:00:00
23
187.5
record: [185.5]
2017-02-10 00:00:00 K值為: 46.06481481481484 D值為: 48.611111111111136 2017-02-13 00:00:00 K值為: 58.52272727272729 D值為: 50.83473625140294
買了一張股票，股價為: 185.5
剩餘資金為: 814500.0
擁有1張股票
2017-02-14 00:00:00
24
187.5
2017-02-15 00:00:00
25
189.0
2017-02-16 00:00:00
26
189.0
2017-02-17 00:00:00
27
189.5
2017-02-20 00:00:00
28
190.0
2017-02

2017-08-17 00:00:00
149
214.0
2017-08-18 00:00:00
150
212.5
2017-08-21 00:00:00
151
211.5
2017-08-21 00:00:00 K值為: 35.185185185185254 D值為: 35.18518518518525 2017-08-21 00:00:00 K值為: 33.164983164983234 D值為: 36.363636363636424
手上有1張股票可賣
賣出1張股票
股價為: 212.5
擁有資金: 1039000.0
獲利: 39000.0
報酬率: 3.9
[0.45, 0.55, 1.2, 1.35, 1.15, 0.95, 1.45, 1.9, 2.1, 2.25, 2.75, 3.35, 3.35, 3.5, 4.05, 3.9]
2017-08-22 00:00:00
152
214.5
record: [211.5]
2017-08-21 00:00:00 K值為: 33.164983164983234 D值為: 36.363636363636424 2017-08-22 00:00:00 K值為: 45.62289562289569 D值為: 37.99102132435471
買了一張股票，股價為: 211.5
剩餘資金為: 827500.0
擁有1張股票
2017-08-23 00:00:00
153
215.5
2017-08-24 00:00:00
154
217.0
2017-08-25 00:00:00
155
217.5
2017-08-24 00:00:00 K值為: 91.16161616161622 D值為: 67.9012345679013 2017-08-25 00:00:00 K值為: 97.22222222222229 D值為: 85.10101010101016
手上有1張股票可賣
賣出1張股票
股價為: 217.0
擁有資金: 1044500.0
獲利: 44500.0
報酬率: 4.45
2017-08-28 00:00:00
156
216.0
2017-08-29 00:00:00
157
216.0
2017-08-30 00:00:00
158
217.5
2017-08-31 00:00:00


255.0
2018-03-16 00:00:00
289
255.0
2018-03-19 00:00:00
290
255.0
2018-03-20 00:00:00
291
253.0
2018-03-21 00:00:00
292
252.5
2018-03-22 00:00:00
293
251.5
2018-03-23 00:00:00
294
245.0
2018-03-26 00:00:00
295
243.5
2018-03-27 00:00:00
296
251.0
record: [243.5]
2018-03-26 00:00:00 K值為: 13.280170122275388 D值為: 23.07333991544517 2018-03-27 00:00:00 K值為: 25.454545454545457 D值為: 20.224298119034952
買了一張股票，股價為: 243.5
剩餘資金為: 856500.0
擁有1張股票
2018-03-28 00:00:00
297
245.0
2018-03-27 00:00:00 K值為: 25.454545454545457 D值為: 20.224298119034952 2018-03-28 00:00:00 K值為: 31.278300243817487 D值為: 23.33767194021276
手上有1張股票可賣
賣出1張股票
股價為: 251.0
擁有資金: 1107500.0
獲利: 107500.0
報酬率: 10.75
2018-03-29 00:00:00
298
244.0
2018-03-30 00:00:00
299
246.0
2018-04-02 00:00:00
300
246.5
record: [246.0]
2018-03-30 00:00:00 K值為: 25.10946907498631 D值為: 29.85934882486605 2018-04-02 00:00:00 K值為: 29.409171075837737 D值為: 29.2363057688728
買了一張股票，股價為: 246.0
剩餘資金為: 861500.0
擁有1張股票
2018-04-03 00:00:00
301
244.0
2018-04-09 00:00:00


賣出1張股票
股價為: 263.5
擁有資金: 1172000.0
獲利: 172000.0
報酬率: 17.2
2018-09-27 00:00:00
422
265.0
2018-09-28 00:00:00
423
262.5
2018-10-01 00:00:00
424
263.0
2018-10-02 00:00:00
425
257.5
2018-10-03 00:00:00
426
260.0
2018-10-04 00:00:00
427
254.0
2018-10-05 00:00:00
428
250.0
2018-10-08 00:00:00
429
243.5
2018-10-09 00:00:00
430
244.0
record: [243.5]
2018-10-08 00:00:00 K值為: 6.190476190476197 D值為: 11.040564373897672 2018-10-09 00:00:00 K值為: 10.190476190476197 D值為: 10.11640211640208
買了一張股票，股價為: 243.5
剩餘資金為: 928500.0
擁有1張股票
2018-10-11 00:00:00
431
227.5
2018-10-11 00:00:00 K值為: 10.190476190476197 D值為: 10.11640211640208 2018-10-11 00:00:00 K值為: 7.760683760683768 D值為: 8.047212047212012
手上有1張股票可賣
賣出1張股票
股價為: 244.0
擁有資金: 1172500.0
獲利: 172500.0
報酬率: 17.25
[0.45, 0.55, 1.2, 1.35, 1.15, 0.95, 1.45, 1.9, 2.1, 2.25, 2.75, 3.35, 3.35, 3.5, 4.05, 3.9, 4.45, 4.4, 4.6, 4.65, 4.95, 5.6, 5.95, 6.1, 6.65, 7.15, 7.2, 7.3, 7.95, 8.8, 8.15, 9.15, 10.0, 10.75, 10.65, 11.3, 11.8, 12.55, 12.55, 13.05, 13.25, 13.35, 13.

2019-05-27 00:00:00 K值為: 11.463693539165211 D值為: 9.667958410096666 2019-05-27 00:00:00 K值為: 6.060606060606035 D值為: 8.764796815111174
手上有1張股票可賣
賣出1張股票
股價為: 233.0
擁有資金: 1250500.0
獲利: 250500.0
報酬率: 25.05
[0.45, 0.55, 1.2, 1.35, 1.15, 0.95, 1.45, 1.9, 2.1, 2.25, 2.75, 3.35, 3.35, 3.5, 4.05, 3.9, 4.45, 4.4, 4.6, 4.65, 4.95, 5.6, 5.95, 6.1, 6.65, 7.15, 7.2, 7.3, 7.95, 8.8, 8.15, 9.15, 10.0, 10.75, 10.65, 11.3, 11.8, 12.55, 12.55, 13.05, 13.25, 13.35, 13.85, 13.95, 15.25, 15.75, 15.5, 16.0, 16.65, 17.2, 17.25, 18.2, 19.5, 20.0, 20.3, 20.95, 21.7, 21.9, 22.1, 22.4, 22.5, 23.15, 23.65, 23.7, 24.35, 24.95, 24.9, 25.5, 25.15, 25.05]
2019-05-28 00:00:00
580
230.5
2019-05-29 00:00:00
581
229.5
2019-05-30 00:00:00
582
231.0
record: [229.5]
2019-05-29 00:00:00 K值為: 6.157731157731132 D值為: 6.377881377881272 2019-05-30 00:00:00 K值為: 12.97591297591295 D值為: 8.682983682983577
買了一張股票，股價為: 229.5
剩餘資金為: 1021000.0
擁有1張股票
2019-05-31 00:00:00
583
235.5
2019-06-03 00:00:00
584
238.0
2019-05-31 00:00:00 K值為: 33.10